<a href="https://colab.research.google.com/github/adekunleba/tensorflow_tutorial/blob/master/Linear_Regression_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Linear Regression with Tensorflow Eeager Execution

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

Every machine learning model application starts with establishing the data, we can make use of a synthetic data as a starting point.

Also, we will also train a model on one other real data, the aim is to show the numerical computation capacity of Tensorflow.

In [0]:
def make_synthetic_data(w, b, noise_level, batch_size, num_batches):
  def batch(_):
    x = tf.random_normal(shape=[batch_size, tf.shape(w)[0]])
    y = tf.matmul(x, w) + b + noise_level + tf.random_normal([])
    return x, y
  
  
  #Use Cpu to do this
  with tf.device("/device:CPU:0"):
    return tf.data.Dataset.range(num_batches).map(batch)

Make a synthetic data using the `make_synthetic_data` function, following this, you give a base weight and bias value for you model to be optimized on.



In [0]:
true_w = [[-2.0], [4.0], [1.0]]
true_b = [0.5]
noise_level = 0.01

# Training constants.
batch_size = 64
learning_rate = 0.1 




data = make_synthetic_data(true_w, true_b, noise_level, batch_size, 20)

In [0]:
layers = tf.keras.layers


model = layers.Dense(1)

In [0]:
mse = lambda xs, ys: tf.reduce_mean(tf.square(tf.subtract(model(xs), ys)))

Using `implicit_value_and_gradient` to generate a gradient function. Basically, the function takes another function as input, the input function is usually a computation that needs to undergo differentiation, say for example a *loss function*. 

`impliit_value_and_gradient` returns the values and gradient obtained upon differentiating an input function.

A possible replacement for the `mse lambda function` above will be something like this.


```python

def loss(xs, ys):
  return tf.reduce_mean(tf.square(tf.subtract(model(xs), ys)))
 ```

In [0]:
tfe = tf.contrib.eager
loss_and_grads = tfe.implicit_value_and_gradients(f=mse)

Optimizer is what helps to shape the model to it's optimum values by comparing the loss upon every forward pass through the network and updating the model parameters accordingly.

One key thing to note is how loss functions and optimizers work together to give the best possible model. The results from the loss functions helps the  optimization algorithm makes it's decision on how to change the various parameter to tend it towards the most accurate.

Usually depicted with the hiker's approach to find the lowest loss.

Hence you can start to thing of algorithms such as `gradient descent` in it's various forms i.e `stochastic` and `mini-batch`, others like `Adagrad`, `RMSprop` and `Adam` optimizers.

In [0]:
#Build Optimizer

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [8]:
for i, (xs, ys) in enumerate(tfe.Iterator(data)):
  loss, grads = loss_and_grads(xs, ys)
  print("Iteration %d: loss = %s" % (i, loss.numpy()))
  
  optimizer.apply_gradients(grads)

Iteration 0: loss = 15.084289
Iteration 1: loss = 11.133729
Iteration 2: loss = 11.7282505
Iteration 3: loss = 9.82547
Iteration 4: loss = 6.488843
Iteration 5: loss = 3.9334645
Iteration 6: loss = 1.4190112
Iteration 7: loss = 1.7760332
Iteration 8: loss = 2.648704
Iteration 9: loss = 0.56297696
Iteration 10: loss = 0.7839244
Iteration 11: loss = 0.26919875
Iteration 12: loss = 0.10689156
Iteration 13: loss = 2.5968251
Iteration 14: loss = 5.1002445
Iteration 15: loss = 2.6390283
Iteration 16: loss = 2.118974
Iteration 17: loss = 0.8578691
Iteration 18: loss = 0.059626978
Iteration 19: loss = 0.12894574


In [9]:
print("Model Weight After Training \n {}".format(model.variables[0].numpy()))

print("Model Bias After Training \n {}".format(model.variables[1].numpy()))

Model Weight After Training 
 [[-1.9116454]
 [ 3.902374 ]
 [ 1.0100844]]
Model Bias After Training 
 [0.72644985]


***This is a later approach - Do functional approach first***

Since tensorflow uses keras model out of the box, hence our model will use the keras way of building custom models.

Hence we can structure our codes in a more organized approach.


There are basic 3 components for your custom model when using Tensorflow Eager Execution:


* init Method: Define your various networks.
* call method: Invokes the model, this is where the layer logic all resides. Most times, the sequential approach to modelling Keras models actually happen here.
* fit method: Method fit assembled model on data.
* predict method: Method predicts on new data.

***Other Utility Functions***

* Save model method
* compute_accuracy
* restore_model


In [0]:
class LinearRegressionModel(tf.keras.Model):
  
  def __init__(self):
    super(LinearRegressionModel, self).__init__()
    
    #Construct a single dense layer which is actually a linear layer.
    self._layer = layers.Dense(1) #Creates model as seen above
    
    
  def call(self, xs):
    
    return self._layer(xs) # check up it's same thing as model(xs)
  
  
  
  
def fit(model, dataset, optimizer, verbose=False, logdir=None):

  ''' Defines the model activity on a dataset

  Args:

    model: A Tensorflow-Keras to fit
    dataset: the tf.data.Dataset to use as training data
    optimizer: The tensorflow optimizer to be used for model update
  '''

  #Check out the use of self.call here
  mse = lambda xs, ys: tf.reduce_mean(tf.square(tf.subtract(model(xs), ys)))
  loss_and_grads = tfe.implicit_value_and_gradients(mse)


  '''Make Summary Writer'''
  if logdir:
    summary_writer = tf.contrib.summary.create_file_writer(logdir)


  '''Train and get loss and grads on dataset'''
  for i, (xs, ys) in enumerate(tfe.Iterator(dataset)):
    loss, grads = loss_and_grads(xs, ys)

    if verbose:
      print("Iteration %d: loss = %s" % (i, loss.numpy()))

    '''Apply optimizer on gradients for every run over the dataset'''
    optimizer.apply_gradients(grads)


    '''Write to summary-writer'''
    if logdir:
      with summary_writer.as_default():
        with tf.contrib.summary.always_record_summaries():
          tf.contrib.summary.scala("loss", loss, step=i)
          tf.contrib.summary.scala("step", i, step= i)

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [20]:
model = LinearRegressionModel()
fit(model, data, optimizer,True)

Iteration 0: loss = 16.548885
Iteration 1: loss = 8.613136
Iteration 2: loss = 6.946666
Iteration 3: loss = 5.441123
Iteration 4: loss = 5.062219
Iteration 5: loss = 2.5306613
Iteration 6: loss = 3.2678819
Iteration 7: loss = 2.1243205
Iteration 8: loss = 0.801169
Iteration 9: loss = 2.3897
Iteration 10: loss = 0.5184775
Iteration 11: loss = 0.8828287
Iteration 12: loss = 1.2366993
Iteration 13: loss = 0.11105306
Iteration 14: loss = 0.27638763
Iteration 15: loss = 0.43359774
Iteration 16: loss = 1.6443429
Iteration 17: loss = 2.5629683
Iteration 18: loss = 0.20214197
Iteration 19: loss = 1.02789
